# CDC variants proportions by state

By [Matt Stiles](https://www.latimes.com/people/matt-stiles) and [Rahul Mukherjee](https://www.latimes.com/people/rahul-mukherjee)

Downloads variant totals and proportions from a [Tableau dashboard](https://covid.cdc.gov/covid-data-tracker/#variant-proportions) published by the U.S. Centers for Disease Control and Prevention.

## Import

Code formatting with [black](https://pypi.org/project/nb-black/).

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


Import dependencies.

In [37]:
import os
import pytz
from datetime import datetime

In [38]:
import pandas as pd
from tableauscraper import TableauScraper as TS

In [39]:
import requests
from bs4 import BeautifulSoup
import json
import re

In [40]:
# !pipenv install tableauscraper=='0.1.10'

In [41]:
tz = pytz.timezone("America/Los_Angeles")

In [42]:
today = datetime.now(tz).date()

## Scrape

Set the URL

In [43]:
url = "https://public.tableau.com/views/WeightedStateVariantTable/StateVBMTable.csv?%3Adisplay_static_image=y&%3AbootstrapWhenNotified=true&%3Aembed=true&%3Alanguage=en-US&:embed=y&:showVizHome=n&:apiID=host0"

In [44]:
src = pd.read_csv(url)

In [45]:
df = src[["State/Jurisdiction", "Variant", "Measure Values"]].copy()

In [46]:
df.rename(
    columns={
        "State/Jurisdiction": "state",
        "Variant": "variable",
        "Measure Values": "value",
    },
    inplace=True,
)

In [47]:
df.variable = (
    df.variable.str.replace(".", "", regex=False)
    .str.replace(r"\(.*\)", "")
    .str.replace(" ", "")
    .str.lower()
)

/var/folders/45/kg6qghdn1sndgj0s14gsfyxh0000gn/T/ipykernel_49657/1757737581.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df.variable.str.replace(".", "", regex=False)


In [48]:
df.variable.unique()

array(['ay1', 'ay2', 'b117', 'b11529', 'b1351', 'b1427/429', 'b1525',
       'b1526', 'b16171', 'b16172', 'b16173', 'b1621', 'other', 'p1',
       'p2'], dtype=object)

In [49]:
df.value = pd.to_numeric(df.value)

In [50]:
df_pivot = df.pivot_table(
    values="value", index="state", columns="variable"
).reset_index()

In [51]:
df_pivot.columns = (
    df_pivot.columns.str.lower()
    .str.replace(".", "", regex=False)
    .str.replace(" ", "_", regex=False)
    .str.replace("/", "_", regex=False)
)

In [52]:
df_pivot["update_date"] = today

In [53]:
df_pivot.head()

variable,state,ay1,ay2,b11529,b117,b1351,b1427_429,b1525,b1526,b16171,b16172,b16173,b1621,other,p1,p2,update_date
0,Alabama,0.000000,0.0,0.716028,0.0,0.0,0.0,0.0,0.0,0.0,0.281818,0.0,0.0,0.007277,0.0,0.0,2022-01-28
1,Alaska,0.000000,0.0,0.453202,0.0,0.0,0.0,0.0,0.0,0.0,0.546798,0.0,0.0,0.000000,0.0,0.0,2022-01-28
2,American Samoa,0.000000,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,2022-01-28
3,Arizona,0.000000,0.0,0.416553,0.0,0.0,0.0,0.0,0.0,0.0,0.578083,0.0,0.0,0.008784,0.0,0.0,2022-01-28
4,Arkansas,0.004288,0.0,0.551982,0.0,0.0,0.0,0.0,0.0,0.0,0.443749,0.0,0.0,0.007178,0.0,0.0,2022-01-28


## Export

Save out the data as a CSV that's datestamped to California time.

In [54]:
data_dir = os.path.join(os.path.abspath(""), "data")

In [55]:
df_pivot.to_csv(
    os.path.join(data_dir, f"variants_cdc_proportions_timeseries_{today}_.csv"),
    index=False,
)